In [1]:
import pandas as pd
import seaborn as sns

GETTING THE WHOLE DATASET

In [3]:
df = pd.read_csv('./WDI_csv/WDIData.csv');

GETTING THE COUNTRIES DATA TO FILTER THE UNWANTED REGIONS(ARAB WORLD, CARRIBBEAN SMALL STATES ETC)

In [4]:
countries_df = pd.read_csv('./WDI_csv/WDICountry.csv');
countries_df.dropna(subset=['Currency Unit' , '2-alpha code', 'WB-2 code'], inplace= True);
countries_df = countries_df[['Country Code', 'Table Name']]

Function to get rid of unwanted countries

In [22]:
def df_by_indicator(indicator):
    return df[ (df['Indicator Code'] == indicator) & (df['Country Code'].isin(countries_df['Country Code']))]

FILTERING THE DATASET FOR POPULATION

In [27]:
df_pop = df_by_indicator('SP.POP.GROW')
df_pop.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
76336,Afghanistan,AFG,Population growth (annual %),SP.POP.GROW,1.816077,1.876528,1.934999,1.992521,2.049423,2.105369,...,2.569114,2.812617,3.095119,3.272703,3.315224,3.183201,2.943234,2.689163,2.490790,NaN
77936,Albania,ALB,Population growth (annual %),SP.POP.GROW,3.015423,3.120855,3.056731,2.953749,2.880686,2.754021,...,-0.673894,-0.496462,-0.269017,-0.165151,-0.183211,-0.207047,-0.291206,-0.159880,-0.091972,NaN
79536,Algeria,DZA,Population growth (annual %),SP.POP.GROW,2.511434,2.485473,2.470737,2.492035,2.560435,2.656201,...,1.720717,1.821358,1.924787,2.006614,2.036092,2.000666,1.919959,1.825463,1.738456,NaN
81136,American Samoa,ASM,Population growth (annual %),SP.POP.GROW,1.403903,2.335966,3.033668,3.558598,3.661243,3.557431,...,-1.418038,-1.054862,-0.571394,-0.162822,0.139320,0.234776,0.180222,0.111575,0.075512,NaN
82736,Andorra,AND,Population growth (annual %),SP.POP.GROW,7.050663,6.941532,6.692697,6.559522,6.241511,5.998800,...,0.714106,-0.015393,-0.829969,-1.588653,-2.013314,-1.956178,-1.537836,-0.944017,-0.409736,NaN


FILTERING THE DATASET FOR GDP

In [29]:
df_gdp = df_by_indicator('NY.GDP.MKTP.KD.ZG')
df_gdp

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
75728,Afghanistan,AFG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,21.020649,8.433290,6.113685,14.434741,3.900575,2.690522,1.310040,2.366712,2.595542,NaN
77328,Albania,ALB,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,3.352610,3.710058,2.550161,1.419968,1.000755,1.774369,2.218752,3.352159,3.841364,NaN
78928,Algeria,DZA,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,-13.605441,-19.685042,34.313729,5.839413,6.206898,...,1.632244,3.634145,2.891866,3.374769,2.767639,3.789121,3.763467,3.300000,1.700000,NaN
80528,American Samoa,ASM,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.237288,0.442478,0.293686,-4.392387,-2.756508,0.944882,1.248050,-2.619414,NaN,NaN
82128,Andorra,AND,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.690654,-5.358826,-4.646543,-1.615218,0.351645,2.277683,0.842204,1.889124,1.873197,NaN
83728,Angola,AGO,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,2.412910,3.452875,3.873331,5.177594,6.842717,4.703971,2.999747,-0.813494,0.721699,NaN
85328,Antigua and Barbuda,ATG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-12.107536,-7.200024,-2.079127,3.506609,-0.103939,5.101940,4.060219,5.342479,3.337049,NaN
86928,Argentina,ARG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,5.427843,-0.852022,-5.308197,10.130298,10.569433,...,-5.918525,10.125398,6.003952,-1.026420,2.405324,-2.512615,2.731160,-1.822542,2.863923,NaN
88528,Armenia,ARM,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-14.149989,2.200000,4.700000,7.200000,3.300000,3.600000,3.200000,0.200000,7.500000,NaN
90128,Aruba,ABW,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.653502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
len(df_pop)

215

In [24]:
df_gni = df_by_indicator('NY.GNP.PCAP.PP.CD')
low_income_countries = df_gni[df_gni['2017'] < 1005]
high_income_countries = df_gni[df_gni['2017'] > 12235]

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
78961,Algeria,DZA,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,12130.0,12580.0,12850.0,13160.0,13420.0,13900.0,14220.0,14870.0,15050.0,NaN
85361,Antigua and Barbuda,ATG,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,19430.0,18310.0,17960.0,18580.0,18980.0,20040.0,20970.0,22090.0,22980.0,NaN
86961,Argentina,ARG,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,16110.0,17710.0,19070.0,19120.0,19730.0,19400.0,19630.0,19260.0,20270.0,NaN
91761,Australia,AUS,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,38820.0,37750.0,40210.0,41430.0,44650.0,45520.0,45270.0,45110.0,47160.0,NaN
93361,Austria,AUT,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,40900.0,42400.0,44600.0,46500.0,48050.0,48850.0,49520.0,50500.0,52500.0,NaN
94961,Azerbaijan,AZE,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,13800.0,14690.0,14680.0,15260.0,16320.0,17100.0,17100.0,16120.0,16650.0,NaN
96561,"Bahamas, The",BHS,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,21760.0,21770.0,22030.0,29630.0,29600.0,29300.0,29650.0,28990.0,29790.0,NaN
98161,Bahrain,BHR,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,35290.0,36080.0,35440.0,36510.0,38840.0,38350.0,44170.0,44010.0,42930.0,NaN
101361,Barbados,BRB,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,15230.0,15730.0,15410.0,16150.0,16270.0,16500.0,16710.0,17170.0,17830.0,NaN
102961,Belarus,BLR,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,14220.0,15600.0,16820.0,17400.0,17620.0,18320.0,17550.0,17250.0,18140.0,NaN
